In [1]:
import tensorflow as tf
import numpy as np
import random
import pickle
import json
import nltk

from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
from keras.layers import Dense,Activation,Dropout
from keras.losses import CategoricalCrossentropy


In [3]:
lemma=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
intents


{'intents': [{'tag': 'greetings',
   'patterns': ['Hi',
    'Hello',
    'Good morning',
    'Good Afternoon',
    'Good Evening',
    'How are you',
    "what's up?"],
   'responses': ['Hello', 'Hey', 'Hi']},
  {'tag': 'name',
   'patterns': ["What's your name",
    'Name',
    'What can I call you',
    'Tell me your name',
    "didn't catch your name",
    'What is your name?'],
   'responses': ['Hi, my name is MediBot',
    'I go by the name MediBot',
    'I am MediBot',
    'I am happy to meet you, I am MediBot']},
  {'tag': 'problem',
   'patterns': ['nose bleeding',
    'nose dripping blood',
    'blood from my nose',
    'blood in nose'],
   'responses': ['Tilt head down and squeeze lightly upper nostrils untils it stops, otherwise seek visit nearest medical centre',
    'Tilt head and squeeze nose strils gently; while seek medical help',
    'MediBot just seeked help, tilt head down and squeeze gently']}]}

In [4]:
words=[]
classes=[]
docs=[]
ignored=['!',',','?','.']

for i in intents['intents']:
    for p in i['patterns']:
        list_of_words=nltk.word_tokenize(p)
        words.extend(list_of_words)#try append
        docs.append(((list_of_words),i['tag']))
        
        if i['tag'] not in classes:
            classes.append(i['tag'])
            
        
        

In [5]:
#store lemma
words=[lemma.lemmatize(word) for word in words if word not in ignored]
words=sorted(set(words))


In [6]:
#save to binary file
pickle.dump(words, open('models/words.plk','wb'))
pickle.dump(classes, open('models/classes.plk','wb'))

In [7]:
train=[]
empty_output= [0]*len(classes) #use numpy to generate zeros

for doc in docs:
    bag=[]
    word_patterns=doc[0]
    word_patterns = [lemma.lemmatize(word.lower()) for word in word_patterns]
    
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    #empty copy of output
    output_row=list(empty_output)
    output_row[classes.index(doc[1])]=1
    train.append([bag,output_row])
random.shuffle(train)
training=np.array(train)




C:\Users\theop\AppData\Local\Temp\ipykernel_14748\4169765869.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(train)


In [8]:
#split it into test and train set

#later on use scikit learn-------------------<
train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [9]:
#sequential ML Model

model=Sequential([
    Dense(units=128,activation='relu'),
    Dense(units=64,activation='relu'),
    Dense(units=len(train_y[0]),activation='softmax')
])

model.compile(loss=CategoricalCrossentropy())
hist=model.fit(np.array(train_x), np.array(train_y),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
4/4 [==============================] - 3s 10ms/step - loss: 1.0811
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9763
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 0.8989
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 0.8335
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 0.7757
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.7229
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.6725
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.6253
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.5751
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5253
Epoch 11/200
4/4 [==============================] - 0s 7ms/step - loss: 0.4812
Epoch 12/200
4/4 [==============================] - 0s 3ms/step - loss: 0.4400
Epoch 13/200
4/4 [==============================] - 0s 3ms/s

In [10]:
model.save("models/chatbotclient.h5", hist)